In [83]:
import gzip
import json

def fix_json_str(js):
    t = "[" + js
    t = t.replace("\n", ",")
    t = t[:-1]
    t += "]"
    return t

In [72]:

def format_number(number):
    if number < 10:
        return "00" + str(number)
    elif number < 100:
        return "0" + str(number)
    else:
        return str(number)

In [73]:
"My name is {}, I'm {}".format("John",36)

'012'

In [76]:
data = []
for i in range(100):
    path = 'downloads/s2c/s2-corpus-{}.gz'.format(format_number(i))
    with gzip.open(path, 'r') as f:
        json_bytes = fix_json_str(f.read().decode('utf-8'))
        data += json.loads(json_bytes)

In [77]:
len(data)

1704534

In [81]:
bio = [d for d in data if "Biology" in d["fieldsOfStudy"]]

In [82]:
len(bio)

120697

In [87]:
import json
import re
from pydash import get


alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

def get_first_sent(abst):
    return get(split_into_sentences(abst), 0)

In [84]:
import pandas as pd
df = pd.DataFrame(bio)
len(df)

In [101]:
df=df.dropna(subset=["paperAbstract"])
df = df[df["paperAbstract"].str.len() > 3]
len(df)

87398

In [102]:
df["first_sent"] = df["paperAbstract"].map(get_first_sent)

In [113]:
dfx = df[["first_sent", "s2Url", "doi", "journalName", "title"]]

In [115]:
dfx.to_csv ("downloads/biology_85k_s2c.csv",index = False, header=True)

In [78]:
FoS = []
for d in data:
    FoS += d["fieldsOfStudy"]

print(set(FoS))

{'Mathematics', 'Geography', 'Political Science', 'Sociology', 'Physics', 'Chemistry', 'History', 'Psychology', 'Philosophy', 'Medicine', 'Environmental Science', 'Geology', 'Business', 'Materials Science', 'Computer Science', 'Art', 'Economics', 'Engineering', 'Biology'}


'ies":[]},'